In [ ]:
import inspect
from unittest import mock

orig_stack = inspect.stack

def stack():
    s = orig_stack()
    i = len(s) - 1
    while i > 0 and not s[i][1].startswith("<ipython-input"):
        i -= 1

    return s[:i + 1]

inspect.stack = stack

with mock.patch('seqtools.errors.inspect', inspect):
    import seqtools

# Error handling and debuging

During the design of a transformation pipeline, mistakes and programming errors are relatively frequent.
SeqTools tries to recover from them and report useful informations as much as possible.
This tutorial reviews some details about the internal error management and should facilitate your debugging sessions.


## Tracing mapping errors

Due to on-demand execution, an error generated by mapping a function to an item won't raise when the mapping is created but rather when the problematic element is read.

In [ ]:
import math
import random
import seqtools

def f1(x):
    return math.sqrt(x)  # this will fail for negative values

data = [0, 4, 6, 7, 2, 4, 4, -1]  # sqrt(-1) raises ValueError

out = seqtools.smap(f1, data)

Due to on-demand execution, no error is raised yet for the last item.

As soon as it is evaluated, SeqTools raises an `EvaluationError` and sets the original exception as its cause.

In [ ]:
list(out)

The `ValueError` that caused the failure is detailed first.

The `EvaluationError` message provides additional clarification: it tells which item caused the error and where the mapping was defined, a crucial debugging information when the mapping function is used at multiple locations in the code.

If you prefer working with the original exception directly and skip the `EvaluationError` wrapper, you can enable the _'passthrough'_ error mode which does just that:

In [ ]:
seqtools.seterr(evaluation='passthrough')

list(out)

In [ ]:
seqtools.seterr(evaluation='wrap')  # revert to normal behaviour

## Errors inside worker

Background workers used by `prefetch` do not share the execution space of the main program and exceptions raised while evaluating elements will happen asynchronously.

To facilitate troubleshooting, SeqTools silently stores exception data, sends it back to the main process to be re-raised when failed items are read.
In practice it looks like exceptions happen when the items are read.

In [ ]:
out = seqtools.smap(f1, data)
out = seqtools.prefetch(out, max_buffered=10)

# evaluate all elements but the last
for i in range(len(out) - 1):
    fast_out[i]

# evaluate the final one
out[-1]

Note that the workers will continue processing other items just fine after an error.

Transfering exceptions back to the parent process has some notable limitations:

- Process-based workers cannot save errors that cannot be pickled, in particular exception types defined inside a function. These will be replaced by an text message.
- Error tracebacks cannot be completely serialized so debuggers won't be able to explore the whole error context.